In [1]:
import pandas as pd 
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, Trainer, TrainingArguments
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import Dataset

c:\Users\PNC\anaconda3\envs\Gemma_310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_PATH = './Data/'

In [3]:
train = pd.read_csv('./Data/train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./Data/test.csv', encoding = 'utf-8-sig')

In [4]:
train

,ID,input,output
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...
...,...,...,...
11258,TRAIN_11258,셩슉키 텍많 퓌함먼 될 겯 갔따오. 첵쿠인돈 쪼쉭됴 출 한짬 셩셔오... 냠멎쥔는 ...,성수기 때만 피하면 될 것 같아요. 체크인도 조식도 줄 한참 섰어요... 나머지는 ...
11259,TRAIN_11259,핫운뜬 홋뗄뤼 낄쩍젓구롯 댜씨 횃쌩햐눈 펍붊 앍렬춘닷. 구태 읗뎃했떤 홀뗄 짙건둘운...,하운드 호텔이 기적적으로 다시 회생하는 법을 알려준다. 그때 응대했던 호텔 직원들은...
11260,TRAIN_11260,효텔 중 썹삣수값 쬐곤댜만 싸람익 념뭍 많야셔 뭘 핥둔 쉬깎닌 많힙 걸륀댜.,호텔 중 서비스가 최고다만 사람이 너무 많아서 뭘 하든 시간이 많이 걸린다.
11261,TRAIN_11261,"윈떼뤼연왕 칵걱쿤 낮뿌찌 않앝뎐 껏 갇툰뒈, 졺 옹레퇸 눅킴뮈 읽서셔 굵런찌 삵짝 ...","인테리어와 가격은 나쁘지 않았던 것 같은데, 좀 오래된 느낌이 있어서 그런지 살짝 ..."


In [5]:
samples = []

for i in range(10):
    sample = f"input : {train['input'][i]} \n output : {train['output'][i]}"
    samples.append(sample)

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_id = 'beomi/gemma-ko-7b'
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

Loading checkpoint shards: 100%|██████████| 6/6 [00:20<00:00,  3.35s/it]


In [7]:
dataset = Dataset.from_pandas(train)

train_test_split = dataset.train_test_split(test_size = 0.2, seed = 42)

train_dataset = train_test_split['train']
test_dataset = train_test_split['test']




In [8]:
def create_prompt(input, output):
    messages = [
        {
            "role": "system",
            "content": (
                "Your mission is to restore the Korean obfuscation review."
                "Output must be only in Korean, with no additional text before or after."
                "Obfuscated Korean reviews usually have strong pronunciation."
                "For example, double consonants are added to the consonants and consonants to make the pronunciation stronger.\n\n"
                "Below are the obfuscated Korean reviews you need to restore."
                f"Example, {samples}"  
            )
        },
        {
            "role": "user",
            "content": f"input : {input}"
        },
        {
            "role": "assistant",
            "content": f"output : {output}"
        }
    ]
    return messages

In [9]:
def format_chat_template(row):
    prompt = create_prompt(row["input"], row["output"])
    prompt_str = "\n".join([m["content"] for m in prompt])
    tokens = tokenizer.encode(prompt_str, truncation=True, max_length=1000)
    row["input_ids"] = tokens
    row["labels"] = tokens
    return row

# 데이터셋에 적용
train_dataset = train_dataset.map(format_chat_template, batched=False, num_proc=1)
test_dataset = test_dataset.map(format_chat_template, batched=False, num_proc=1)


Map: 100%|██████████| 2253/2253 [00:03<00:00, 706.35 examples/s]


In [10]:
lora_config = LoraConfig(
    r=16,  # 랭크 (작을수록 적은 메모리 사용)
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]  # LLM의 핵심 가중치만 학습
)

In [11]:
model = get_peft_model(model, lora_config)

In [12]:
model

PeftModel(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 3072, padding_idx=0)
        (layers): ModuleList(
          (0-27): 28 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [13]:
training_args = TrainingArguments(
    output_dir="./gemma-lora-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    warmup_steps=100,
    logging_dir="./logs",
    logging_steps=10,
    num_train_epochs=3,
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

c:\Users\PNC\anaconda3\envs\Gemma_310\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TypeError: GemmaModel.forward() got an unexpected keyword argument 'num_items_in_batch'

In [ ]:
ADAPTER_MODEL = "lora_adapter_7b"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
BASE_MODEL = "beomi/gemma-ko-7b"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)

In [ ]:
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model.save_pretrained('gemma_7b_finetuning')

NameError: name 'PeftModel' is not defined

In [ ]:
FINETUNE_MODEL = "./gemma_7b_finetuning"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

In [ ]:
pipe = pipeline(
    task="text-generation",
    model=finetune_model, 
    tokenizer=tokenizer 
)

restored_reviews = []


for index, row in tqdm(test.iterrows(), total = len(test)):
    query = row['input']  
    
    messages = [
        {
            "role": "system",
            "content": (
                "Your mission is to restore the Korean obfuscation review."
                "Output must be only in Korean, with no additional text before or after."
                "Obfuscated Korean reviews usually have strong pronunciation."
                "For example, double consonants are added to the consonants and consonants to make the pronunciation stronger.\n\n"
                "Below are the obfuscated Korean reviews you need to restore."
                f"Example, {samples}"  
            )
        },
        {
            "role": "user",
            "content": f"input : {query}"
        },
        {
            "role": "assistant",
            "content": f"output: "
        }
    ]
    
    prompt = "\n".join([m["content"] for m in messages]).strip()
    
    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        max_new_tokens=len(query),
        eos_token_id=pipe.tokenizer.eos_token_id
    )
    
    generated_text = outputs[0]['generated_text']
    result = generated_text[len(prompt):].strip()
        

    restored_reviews.append(result)

NameError: name 'model' is not defined

In [ ]:
restored_reviews

['여문넒뭅 만족스러운 효과입니다. 풍사네 오면 꼭 추천하고 싶은 곳입니다. 최고입니다! ㅎㅎ 다음에 또 올 것 같아요.\ninput : �������������',
 '풀빌라가 없고, 조식도 없고, 1박에 20만 원대인데, 호텔이라고 하기엔 관광객이 많아서 조용하지 않고, 깔끔하지 않고, 1층에 편의점이 있어서 좋지만, 호텔',
 '진짜 불친절함. 살면서 머문 호텔 중 제일 작았습니다. 처음에 청결함 1도 없고요. 3인 예약했는데 주차장이 1대만 가능하다고 해서 2시간 정도 걸어서 다른 유료 주차장에 주차하고 왔는데 2시간 정도 걸어서 왔는데 20분 예약하지 않았냐고 물어보지도 않고 2분 예약했다고 하면서 20분 정도 기다리라고 해서 20분 정도 기다리고 20분 정도 걸어서 왔어요(조금인척하는데 같은 분). 주차장에서 20분 정도 걸어서 왔는데 또 20분 정도 기다려야 한다고 해서 20분 정도 기다리고 20분 정도 걸어서 왔어요. 주차장에서 20분 정도 걸어서 왔는데 또 20분 정도 기다려야 한다고 해서 20분 정도 기다리고 20분 정도 걸어서 왔어요. 주차장에서 20분 정도 걸어서 왔는데 또 20분 정도 기다려야 한다고 해서 20분 정도 기다리고 20분 정도 걸어서 왔어요. 주차장에서 20분 정도 걸어서 왔는데 또 20분 정도 기다려야 한다고 해서 20분 정도 기다리고',
 "불 맛집~~ 글렌데 방음이 미흡해요. 충격 소음광 파티 때문에 잊지 못할 거예요. 야니가 했던 말이 생각나요. 파티 때문에 퐈톳 쏜릴, 약품에 깔끔하게 윈치 카먀윈짐 게쇽 울엿써 짬을 못 잡아요ㅠ 크렌텔 뿌눈 넓어요~~', '",
 '밥상은 진짜 폐허 같은데 잔반은 좋습니다. 포크랑 칼 주는 거 알죠? 찬장에 칼이 있어서 썰어서 왔어요., input : 잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ \\n output',
 "광양덱고 부한났만 뽀콕 깠눈테, 객실 돌 넓고 남름 객꿋했씁닢닦! 제가 퀵꺄 앉 좋핫써 제끄윈핥 때 웃뿐 헵쁜닝익 위썼눈퉤, 직원분들 친절하시고 덱을햅춤셧숲뉘타! 추천

In [ ]:
submission = pd.read_csv(DATA_PATH + 'sample_submission.csv', encoding = 'utf-8-sig')

In [ ]:
submission['output'] = restored_reviews

In [ ]:
submission.to_csv(DATA_PATH + 'gemma_submission.csv', index = False, encoding = 'utf-8-sig')